In [ ]:
import arcpy
import os

bd_trabajo = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\_7_GDBs\Diagnostico_Indicador.gdb"
cmt12 = r"C:\PUBLIC\SERESCO\Resultados\_8_Py\ctm12.prj"
arcpy.env.overwriteOutput=True

arcpy.env.workspace = bd_trabajo

diccionario = {}
for dataset in arcpy.ListDatasets(wild_card='_*'):
    for feature in arcpy.ListFeatureClasses(feature_dataset=dataset):
        if feature == 'Vertimientos':
            for campo in arcpy.ListFields(dataset = feature):
                if campo.type == 'String':
                    if feature not in diccionario:                        
                        diccionario[feature] = [campo.name]
                    else: 
                        diccionario[feature].append(campo.name) 
                else:
                    pass
        else:
            pass
            
resultados = {}

for llave, valor in diccionario.items():    
    with arcpy.da.SearchCursor(llave, valor) as cursor:
        for fila in cursor:
            for i, nombre_campo in enumerate(valor):
                valor_campo = fila[i]  # Obtener el valor del campo en la fila actual
                # ? La sentencia if valor: es una construcción en Python que se utiliza para evaluar si 
                # ? una variable valor contiene un valor considerado "verdadero" o "no vacío"
                if valor_campo:  # Verificar si el valor no es nulo
                    longitud = len(valor_campo) 
                    # ? [if nombre_campo in resultados] verifica si el nombre_campo actual se encuentra ya en el diccionario resultados. 
                    # ? Si el nombre del campo está presente en el diccionario, significa que se ha procesado este campo anteriormente y ya 
                    # ? se tiene un valor de longitud almacenado en resultados.
                    if nombre_campo in resultados:
                        # ? resultados[nombre_campo] se relaciona con el valor de longitud (en diccionario el value)
                        # ? y el condicional compara el valor existente con el nuevo encontrado en el bucle. Lo que hace este
                        # ? campo es llamar el valor que se almacenó previmante en el key (que es el nombre de la capa)
                        if longitud > resultados[nombre_campo]:
                            # ? Si resulta que el nuevo valor es mayor al que ya existia, lo actualiza por el nuevo
                            resultados[nombre_campo] = longitud
                    else:
                        # ? Si el campo no existe en el diccionario lo creo y le asigna su longitud correspondiente.
                        resultados[nombre_campo] = longitud
                        
for dataset in arcpy.ListDatasets(wild_card='_*'):
    for feature in arcpy.ListFeatureClasses(feature_dataset=dataset):
        if feature == 'Vertimientos':            
            for campo in arcpy.ListFields(dataset = feature):
                for llave, valor in resultados.items(): 
                    if campo.name == llave:                        
                        if '_Norm' in campo.name:
                            palabras = campo.name.split('_')
                            nuevo_nombre = '_'.join([palabra.capitalize() for palabra in palabras])
                            arcpy.management.AlterField(feature, field=campo.name, new_field_name=nuevo_nombre[:-5])
                            print(print(f" *** {campo.name} actualizado ***"))
                        else:                            
                            arcpy.management.AddField(feature, str(campo.name)+'_Norm', field_type = 'TEXT', field_length = int(valor))
                            print(f" 1. Se crea campo {str(campo.name)+'_Norm'} para la capa {feature}")
                            arcpy.management.CalculateField(feature, str(campo.name)+'_Norm', "!"+campo.name+"!", 'PYTHON3')
                            print(f" 2. Se se migra {campo.name} para la capa {feature}")
                            arcpy.management.DeleteField(feature, campo.name)
                            print(f" 3. Se borra campo {campo.name} de capa {feature}")
                            # ? Generar Letra Capital en todas las palabras que conforman el nombre de la capa
                            palabras = campo.name.split('_')
                            nuevo_nombre = '_'.join([palabra.capitalize() for palabra in palabras])
                            print(str(campo.name)+'_Norm', nuevo_nombre)
                            arcpy.management.AlterField(feature, field=str(campo.name)+'_Norm', new_field_name=nuevo_nombre, new_field_alias=nuevo_nombre)
                            print(f" 4. Se actualiza campo {campo.name} de capa {feature}")                  
                            
                            